In [1]:
import numpy as np
import nibabel as nb
from scipy.spatial.transform import Rotation as rotation

## Helper functions

In [2]:
# calc_transforms():

# Given
#   - A nibable img as returned by nb.load()
# Compute
#   - The vox2ras matrix (this is simply img.affine)
#   - The grad2ras matrix (mri_info calls this xform info)
#   - See mri.cpp:12818
def calc_transforms(img):
    vox2ras = np.copy(img.affine)
    grad2ras = np.copy(vox2ras)

    # - pixdims is same as xsize, ysize, zsize in mri.cpp:12818 
    # - c is same as c_r, c_a, c_s in mri.cpp:12818
    # - use of 1:4 here is probably fragile
    pixdims = img.header['pixdim'][1:4]
    c = img.header['dim'][1:4] / 2

    grad2ras[0:3,0:3] = vox2ras[0:3,0:3] / pixdims
    grad2ras[0:3,3]   = vox2ras[0:3,3]   + vox2ras[0:3,0:3] @ c

    return vox2ras, grad2ras

In [8]:
# calc_aa():

# Given:
#  - An image taken with a particular FOV: which gives
#    - V2R: A VOX2RAS matrix (4x4)
#    - G2R: A GRAD2RAS matrix (4x4; xform info)
#  - An alignment vector (along the 0th, 1st or 2nd dimension)
#  - A polarity (+1 or -1)
#  - 2 voxel coordinates of that image;
#    - p1_vox: where you'd like the origin for the new FOV to be (4x1)
#    - p2_vox: a point you would like alinged with the alignment vector (4x1)
#    
# Compute:
#  - A transformation that when applied to G2R (G2R * T or T * G2R?) results in
#    - T_ras2ras: A RAS2RAS trasform (4x4), such that
#      - T * p1_ras = [0,0,0,1]
#      - T * p2_ras is aligned with the alignment vector 
#    - T_lps2lps: Same transform, but in lps2lps
#
# We know
#  - R2L: A RAS to LPS conversion matrix
#  - We can convert points or vectors:
#    - p_ras = VOX2RAS * p_vox
#    - p_vox = inv(VOX2RAS) * p_ras
#  - There are infite translations T that exits, we aribtrarily pick one
#    - There's probably a smarter way to constrain to get a unique solution

def calc_aa(img_filename, p1_vox4, p2_vox4, align_dim=0, polarity=1):

    # converts from ras2lps and vice versa (note that R2L == inv(R2L) )
    R2L            = np.array([[-1,  0, 0, 0],
                               [ 0, -1, 0, 0],
                               [ 0,  0, 1, 0],
                               [ 0,  0, 0, 1]], dtype=np.float64)
    polarity_flip3 = np.array([[-1,  0, 0],
                               [ 0, -1, 0],
                               [ 0,  0, -1]], dtype=np.float64)

    img = nb.load(img_filename)
    V2R, G2R = calc_transforms(img)
    
    p1_ras4 = V2R @ p1_vox4
    p2_ras4 = V2R @ p2_vox4

    p1_ras3 = p1_ras4[0:3]
    p2_ras3 = p2_ras4[0:3]
    
    # The primary vector to align with
    q1_ras3 = (p2_ras3 - p1_ras3) / np.linalg.norm(p2_ras3 - p1_ras3)
    if polarity < 0:
        q1_ras3 = polarity_flip3 @ q1_ras3
    
    # Find other 2 arbitrary (but deterministic) ortho vectors
    # to build our space
    # This wont work if the vector we cross with is the zero vector,
    # so choose the point with the larger norm as the arb vector
    if (np.linalg.norm(p1_ras3) > np.linalg.norm(p2_ras3)):
        q2_ras3 = np.cross(q1_ras3.T, p1_ras3.T).T / np.linalg.norm(np.cross(q1_ras3.T, p1_ras3.T).T)
    else:
        q2_ras3 = np.cross(q1_ras3.T, p2_ras3.T).T / np.linalg.norm(np.cross(q1_ras3.T, p2_ras3.T).T)
    q3_ras3 = np.cross(q1_ras3.T, q2_ras3.T).T
    
    if align_dim == 0:
        rot = np.linalg.inv(np.hstack((q1_ras3,q2_ras3,q3_ras3)))
    elif align_dim == 1:
        rot = np.linalg.inv(np.hstack((q2_ras3,q1_ras3,q3_ras3)))
    else:
        rot = np.linalg.inv(np.hstack((q2_ras3,q3_ras3,q1_ras3)))
    
    close_enough = 1e-12
    if (np.linalg.det(rot) - 1.0) > close_enough:
        print(f"WARNING: det(rot) is {np.linalg.det(rot)}, should be 1.0", )
    
    trans = -1 * (rot @ p1_ras3)
    T_ras2ras = np.eye(4)
    T_ras2ras[0:3,0:3] = rot
    T_ras2ras[0:3,3] = trans.ravel()
    
    # See registered_image.py:133
    #    `T = np.dot(np.dot(flip, T), flip)`
    # Note that registered_image.py:133 only works becuase flip==inv(flip)
    # here we use the more general formula
    T_lps2lps = R2L @ T_ras2ras @ np.linalg.inv(R2L)
    
    return T_ras2ras, T_lps2lps

In [13]:
# Convert a 4x4 numpy matrix representing a transform to string
# To be used with the --trans flag of auto_register.py
def trans2str(trans):
    close_enough = 1e-12
    if (np.linalg.det(trans[0:3,0:3]) - 1.0) > close_enough:
        print(f"WARNING: det(trans[0:3,0:3]) is {np.linalg.det(trans[0:3,0:3])}, should be 1.0")

    trans_1d = trans.ravel()
    trans_str = ''
    for i in trans_1d:
        trans_str += str(i) + ' '
    return trans_str

In [20]:
# Wrapper around calc_aa() for all possible dim/polarity combinations
def calc_all_aas(img_filename, p1_vox4, p2_vox4):
    dim_polarity = np.array([[0, 1],
                             [0,-1],
                             [1, 1],
                             [1,-1],
                             [2, 1],
                             [2,-1]], dtype=np.int32)
    
    for i in range(dim_polarity.shape[0]):
        dim=dim_polarity[i,0]
        pol=dim_polarity[i,1]
        print(f'Running with dim={dim} and pol={pol}:')
        T_r2r, T_l2l = calc_aa(img_filename, p1_vox, p2_vox, align_dim=dim, polarity=pol)
        T_rot = rotation.from_matrix(T_r2r[0:3,0:3])
        T_angle = np.linalg.norm(T_rot.as_rotvec(degrees=True))
        # This is the string we will pass to autoregister.  Note we need to send the inverse of the transform
        # We've computed
        T_l2l_string = trans2str(np.linalg.inv(T_l2l))
        print('T_r2r:')
        print(T_r2r)
        print('T_l2l:')
        print(T_l2l)
        print('T_l2l as a string (for `auto_register.py -trans`)')
        print(T_l2l_string)
        print(f'The angle of T is {T_angle} degrees')
        print('-------------------------------------------------------')

## Start Here

## Saggital tests

- Scout is series 13 of `areg20230509`
- This was sent to ext pc using vsend_20230417
- Corresponds to `/home/paul/lcn/20230509-areg-bay1-arbitrarty-transform/img-00000.nii.gz`

In [21]:
# This file was sent with vsend_20230417 using the following to receive it:
# ```
# /home/paul/cmet/git/areg/auto_register-synthstrip/src/auto_register.py \
#   -s /home/paul/lcn/20230509-areg-bay1-arbitrarty-transform \
#   -H 0.0.0.0 \
#   -f
# ```
# It is a saggital aquisition at isocenter with 4mm voxels
img_filename = '/home/paul/lcn/20230509-areg-bay1-arbitrarty-transform/img-00000.nii.gz'

# The above image was opened in freeview to estimate the locations of the 2 vitamine E
# capsules

# Capsule on nose ridge
p1_vox = np.array([[14, 36, 31, 1]]).T

# Capsule on head
p2_vox = np.array([[27, 15, 21, 1]]).T

In [22]:
calc_all_aas(img_filename,p1_vox,p2_vox)

Running with dim=0 and pol=1:
T_r2r:
[[ 3.75293313e-01 -4.87881306e-01  7.88115956e-01  4.69867227e+01]
 [-8.60255637e-01  1.33260459e-01  4.92140111e-01  3.55271368e-15]
 [-3.45130655e-01 -8.62678087e-01 -3.69690885e-01  5.68880294e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_l2l:
[[ 3.75293313e-01 -4.87881306e-01 -7.88115956e-01 -4.69867227e+01]
 [-8.60255637e-01  1.33260459e-01 -4.92140111e-01 -3.55271368e-15]
 [ 3.45130655e-01  8.62678087e-01 -3.69690885e-01  5.68880294e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_l2l as a string (for `auto_register.py -trans`)
0.3752933125204009 -0.8602556369345474 0.3451306545350628 -2.0000000000000036 -0.4878813062765211 0.1332604590956891 0.8626780865580865 -72.00000000000001 -0.7881159562928417 -0.4921401113137825 -0.3696908847573572 -16.0 0.0 0.0 0.0 1.0 
The angle of T is 115.50364757981136 degrees
-------------------------------------------------------
Running with dim=0 and pol=-1

In [ ]:
# Ignore series 16, I realised I had to send the inverse of the transform derived..

In [ ]:
# I then sent the T_l2l for dim=0; pol=1
# The resulting image after sending that transform is series 19 (gre_aa_sag) and this should have
# p1 in the center of the fov and p2 along the x axis (positive dir)

In [ ]:
# I then sent the T_l2l for dim=1; pol=1
# The resulting image after sending that transform is series 22 (gre_aa_sag) and this should have
# p1 in the center of the fov and p2 along the y axis (positive dir)

In [ ]:
# I then sent the T_l2l for dim=2; pol=1
# The resulting image after sending that transform is series 25 (gre_aa_sag) and this should have
# p1 in the center of the fov and p2 along the z axis (positive dir)

## Coronal tests

In [23]:
img_filename = '/home/paul/lcn/20230509-areg-bay1-arbitrarty-transform/img-00001.nii.gz'

# The above image was opened in freeview to estimate the locations of the 2 vitamine E
# capsules

# Capsule on nose ridge
p1_vox = np.array([[31, 35, 13, 1]]).T

# Capsule on head
p2_vox = np.array([[22, 14, 26, 1]]).T

calc_all_aas(img_filename,p1_vox,p2_vox)

Running with dim=0 and pol=1:
T_r2r:
[[ 3.42376136e-01 -4.94543307e-01  7.98877650e-01  4.48132320e+01]
 [-8.82974215e-01  1.21266845e-01  4.53487472e-01  8.88178420e-16]
 [-3.21146566e-01 -8.60651654e-01 -3.95150115e-01  6.02310073e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_l2l:
[[ 3.42376136e-01 -4.94543307e-01 -7.98877650e-01 -4.48132320e+01]
 [-8.82974215e-01  1.21266845e-01 -4.53487472e-01 -8.88178420e-16]
 [ 3.21146566e-01  8.60651654e-01 -3.95150115e-01  6.02310073e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_l2l as a string (for `auto_register.py -trans`)
0.34237613560884006 -0.8829742151587887 0.32114656643077827 -3.999999999999996 -0.49454330699054677 0.12126684500034868 0.860651654163585 -74.0 -0.7988776497539603 -0.4534874724492205 -0.3951501145833144 -12.000000000000007 0.0 0.0 0.0 1.0 
The angle of T is 117.75900251697126 degrees
-------------------------------------------------------
Running with dim=0 and pol

In [ ]:
# I then sent the T_l2l for dim=0; pol=-1
# The resulting image after sending that transform is series 29 (gre_aa_cor) and this should have
# p1 in the center of the fov and p2 along the x axis (negative dir)

In [ ]:
# I then sent the T_l2l for dim=1; pol=-1
# The resulting image after sending that transform is series 32 (gre_aa_cor) and this should have
# p1 in the center of the fov and p2 along the y axis (negative dir)

In [ ]:
# I then sent the T_l2l for dim=2; pol=-1
# The resulting image after sending that transform is series 35 (gre_aa_cor) and this should have
# p1 in the center of the fov and p2 along the z axis (negative dir)

## Transverse tests

In [24]:
img_filename = '/home/paul/lcn/20230509-areg-bay1-arbitrarty-transform/img-00002.nii.gz'

# The above image was opened in freeview to estimate the locations of the 2 vitamine E
# capsules

# Capsule on nose ridge
p1_vox = np.array([[31, 36, 13, 1]]).T

# Capsule on head
p2_vox = np.array([[22, 14, 26, 1]]).T

calc_all_aas(img_filename,p1_vox,p2_vox)

Running with dim=0 and pol=1:
T_r2r:
[[ 3.32196062e-01 -4.79838756e-01  8.12034818e-01  4.71718408e+01]
 [-8.83070233e-01  1.44276263e-01  4.46510160e-01 -2.66453526e-15]
 [-3.31410229e-01 -8.65412692e-01 -3.75803315e-01  5.93533271e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_l2l:
[[ 3.32196062e-01 -4.79838756e-01 -8.12034818e-01 -4.71718408e+01]
 [-8.83070233e-01  1.44276263e-01 -4.46510160e-01  2.66453526e-15]
 [ 3.31410229e-01  8.65412692e-01 -3.75803315e-01  5.93533271e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_l2l as a string (for `auto_register.py -trans`)
0.33219606173650296 -0.8830702331592302 0.33141022898344613 -4.000000000000003 -0.47983875584161545 0.14427626344573352 0.8654126924181942 -74.0 -0.8120348175781186 -0.4465101601467093 -0.3758033154811595 -15.999999999999995 0.0 0.0 0.0 1.0 
The angle of T is 116.72222449304087 degrees
-------------------------------------------------------
Running with dim=0 and po

In [ ]:
# I then sent the T_l2l for dim=0; pol=1
# The resulting image after sending that transform is series 39 (gre_aa_tra) and this should have
# p1 in the center of the fov and p2 along the x axis (pos dir)

In [ ]:
# I then sent the T_l2l for dim=1; pol=1
# The resulting image after sending that transform is series 42 (gre_aa_tra) and this should have
# p1 in the center of the fov and p2 along the y axis (pos dir)

In [ ]:
# I then sent the T_l2l for dim=1; pol=1
# The resulting image after sending that transform is series 45 (gre_aa_tra) and this should have
# p1 in the center of the fov and p2 along the z axis (pos dir)